In [1]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

In [2]:
from examples.models.resnet import resnet18

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
threads = 16
torch.set_num_threads(threads)

#maybe better performance (Jupyter)
#%env OMP_PLACES=cores
#%env OMP_PROC_BIND=close
#%env OMP_WAIT_POLICY=active

#WSL
os.environ['OMP_PLACES'] = 'cores'
os.environ['OMP_PROC_BIND'] = 'close'
os.environ['OMP_WAIT_POLICY'] = 'active'

In [4]:
#axx_mult = 'mul8s_acc'


In [5]:
axx_mult = 'exact'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_exact/build.ninja...
Building extension module PyInit_conv2d_exact...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading exten

ResNet(
  (conv1): AdaPT_Conv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): AdaPT_Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
        (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

In [6]:
import torch
from torchvision import transforms as T
from torchvision.datasets import STL10
from torch.utils.data import DataLoader


def val_dataloader(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = STL10(root="datasets/stl10_data", split="test", download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=32,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader


def calculate_mean_std(dataset):

    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    for images, _ in loader:
        images = images.view(images.size(0), images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    mean /= len(dataset)
    std /= len(dataset)
    return mean, std


transform = T.Compose(
    [
        T.RandomCrop(96, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
    ]
)

dataset = STL10(root="datasets/stl10_data", split="train", download=True, transform=transform)

mean, std = calculate_mean_std(dataset)

transform = T.Compose(
    [
        T.RandomCrop(96, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean, std),
    ]
)


dataset = STL10(root="datasets/stl10_data", split="train", download=False, transform=transform)


evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)


data = val_dataloader(mean, std)


data_t = DataLoader(trainset_1, batch_size=64, shuffle=False, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


In [7]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)

100%|██████████| 2/2 [00:04<00:00,  2.45s/it]
W0222 18:59:29.298371 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.298740 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.298977 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.299181 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.299377 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.299570 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.299765 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.299949 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.300144 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.300338 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:29.300535 140027934322880 tensor

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.5903 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6190 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3132 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.4999 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

In [8]:
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

100%|█████████████████████████████████████████████████████████████████████████████████| 250/250 [26:03<00:00,  6.25s/it]

1563.7572889110015
Accuracy of the network on the 10000 test images: 30.1625 %


In [9]:
axx_mult = 'appro1'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro1/build.ninja...
Building extension module PyInit_conv2d_appro1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loadi

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.27s/it]
W0127 17:58:22.760195 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.760936 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.761435 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.761948 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.762579 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.763264 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.763914 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.764824 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.765747 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 17:58:22.766747 140173119096640 tensor_quantizer.py:17

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.5911 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6151 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3165 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.4968 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|█████████████████████████████████████████████████████████████████████████████████| 250/250 [24:27<00:00,  5.87s/it]


1467.5419439399993
Accuracy of the network on the 10000 test images: 12.8000 %
Epoch 1/15
Epoch: [0]  [0/8]  eta: 0:03:20  lr: 0.0001  img/s: 2.5626608683080474  loss: 2.1491 (2.1491)  acc1: 32.8125 (32.8125)  acc5: 71.8750 (71.8750)  time: 25.0021  data: 0.0281
Epoch: [0]  [1/8]  eta: 0:03:00  lr: 0.0001  img/s: 2.407788637427607  loss: 2.0949 (2.1220)  acc1: 32.8125 (32.8125)  acc5: 71.8750 (76.5625)  time: 25.8086  data: 0.0313
Epoch: [0]  [2/8]  eta: 0:02:38  lr: 0.0001  img/s: 2.3360421490263303  loss: 2.0949 (2.0397)  acc1: 32.8125 (35.4167)  acc5: 81.2500 (78.1250)  time: 26.3494  data: 0.0323
Epoch: [0]  [3/8]  eta: 0:02:13  lr: 0.0001  img/s: 2.328649485527525  loss: 2.0949 (2.0949)  acc1: 32.8125 (33.9844)  acc5: 71.8750 (76.5625)  time: 26.6411  data: 0.0324
Epoch: [0]  [4/8]  eta: 0:01:46  lr: 0.0001  img/s: 2.455134510887385  loss: 2.0949 (2.0512)  acc1: 32.8125 (35.9375)  acc5: 73.4375 (75.9375)  time: 26.5335  data: 0.0330
Epoch: [0]  [5/8]  eta: 0:01:19  lr: 0.0001  img

100%|█████████████████████████████████████████████████████████████████████████████████| 250/250 [23:46<00:00,  5.71s/it]

1426.6530973600165
Accuracy of the network on the 10000 test images: 38.9500 %


In [10]:
axx_mult = 'appro2'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro2/build.ninja...
Building extension module PyInit_conv2d_appro2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro2...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro2, skipping build step...
Loading extension module PyInit_conv2d_appro2...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro2, skipping build step...
Loading extension module PyInit_conv2d_appro2...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro2, skipping build step...
Loadi

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.07s/it]
W0127 19:39:26.841186 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.842470 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.842966 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.843447 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.844125 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.844841 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.846164 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.846925 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.847591 140173119096640 tensor_quantizer.py:173] Disable HistogramCalibrator
W0127 19:39:26.848444 140173119096640 tensor_quantizer.py:17

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.5911 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6091 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3120 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5012 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|█████████████████████████████████████████████████████████████████████████████████| 250/250 [24:09<00:00,  5.80s/it]


1449.5421034650062
Accuracy of the network on the 10000 test images: 10.0000 %
Epoch 1/15
Epoch: [0]  [0/8]  eta: 0:03:15  lr: 0.0001  img/s: 2.6233859169991365  loss: 2.3461 (2.3461)  acc1: 12.5000 (12.5000)  acc5: 54.6875 (54.6875)  time: 24.4357  data: 0.0397
Epoch: [0]  [1/8]  eta: 0:03:00  lr: 0.0001  img/s: 2.3686947424807823  loss: 2.3086 (2.3274)  acc1: 12.5000 (15.6250)  acc5: 50.0000 (52.3438)  time: 25.7530  data: 0.0455
Epoch: [0]  [2/8]  eta: 0:02:37  lr: 0.0001  img/s: 2.3404308043677378  loss: 2.3461 (2.3549)  acc1: 12.5000 (14.0625)  acc5: 50.0000 (50.0000)  time: 26.2973  data: 0.0438
Epoch: [0]  [3/8]  eta: 0:02:12  lr: 0.0001  img/s: 2.336144836608688  loss: 2.3137 (2.3446)  acc1: 12.5000 (14.4531)  acc5: 50.0000 (52.3438)  time: 26.5796  data: 0.0406
Epoch: [0]  [4/8]  eta: 0:01:47  lr: 0.0001  img/s: 2.3298520128356155  loss: 2.3461 (2.3506)  acc1: 12.5000 (12.5000)  acc5: 51.5625 (52.1875)  time: 26.7651  data: 0.0400
Epoch: [0]  [5/8]  eta: 0:01:20  lr: 0.0001  i

100%|█████████████████████████████████████████████████████████████████████████████████| 250/250 [25:19<00:00,  6.08s/it]

1519.267150871019
Accuracy of the network on the 10000 test images: 9.7500 %


In [8]:
axx_mult = 'appro3'
model = resnet18(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro3/build.ninja...
Building extension module PyInit_conv2d_appro3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loadi

100%|██████████| 2/2 [00:04<00:00,  2.45s/it]
W0222 18:59:41.352475 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.353087 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.353557 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.353981 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.354461 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.354884 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.356292 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.356751 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.357200 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.357582 140027934322880 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 18:59:41.358002 140027934322880 tensor

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.5903 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1513 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6102 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0620 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3062 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0380 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5005 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0354 calibrator=HistogramCalibrator quant)
layer1.1

100%|██████████| 250/250 [06:07<00:00,  1.47s/it]


367.93398952670395
Accuracy of the network on the 10000 test images: 10.0000 %
Epoch 1/15
Epoch: [0]  [0/8]  eta: 0:01:25  lr: 0.0001  img/s: 6.000115292499507  loss: 2.0273 (2.0273)  acc1: 32.8125 (32.8125)  acc5: 73.4375 (73.4375)  time: 10.6880  data: 0.0216
Epoch: [0]  [1/8]  eta: 0:01:14  lr: 0.0001  img/s: 6.090958676034975  loss: 2.0273 (2.0401)  acc1: 29.6875 (31.2500)  acc5: 73.4375 (75.7812)  time: 10.6089  data: 0.0220
Epoch: [0]  [2/8]  eta: 0:01:03  lr: 0.0001  img/s: 6.117231697513587  loss: 2.0273 (2.0067)  acc1: 32.8125 (32.2917)  acc5: 78.1250 (78.6458)  time: 10.5675  data: 0.0221
Epoch: [0]  [3/8]  eta: 0:00:52  lr: 0.0001  img/s: 6.112930398837203  loss: 2.0273 (2.0228)  acc1: 31.2500 (32.0312)  acc5: 73.4375 (77.3438)  time: 10.5486  data: 0.0222
Epoch: [0]  [4/8]  eta: 0:00:42  lr: 0.0001  img/s: 6.167621200552706  loss: 2.0273 (1.9993)  acc1: 32.8125 (33.4375)  acc5: 75.0000 (76.8750)  time: 10.5187  data: 0.0222
Epoch: [0]  [5/8]  eta: 0:00:31  lr: 0.0001  img/s

100%|██████████| 250/250 [05:53<00:00,  1.41s/it]

353.4519580937922
Accuracy of the network on the 10000 test images: 38.0125 %
